In [4]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import gzip

In [5]:
def load_data(file_path):
    with gzip.open(file_path, 'rt') as f:
        data = np.loadtxt(f)
    labels = data[:, 0].astype(int)
    images = data[:, 1:].reshape(-1, 16, 16) / 255.0  # Normalize pixel values
    return images, labels